In [122]:
library(TCGAbiolinks)

In [123]:
query.legacy <- GDCquery(project = "TCGA-COAD",  ## 肿瘤类型，可以修改
                         data.category = "Clinical", ##数据分类为临床
                         data.type = "Pathology report", ## 数据类型为病理学报告
                         legacy = TRUE, 
                         barcode = c("TCGA-RU-A8FL","TCGA-AA-3972") ##制定患者编号
                         ) 

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg19

Warning message in open.connection(con, "rb"):
"cannot open URL 'https://api.gdc.cancer.gov/legacy/projects/TCGA-COAD?expand=summary,summary.data_categories&pretty=true': HTTP status was '410 Gone'"


ERROR: Error in open.connection(con, "rb"): cannot open the connection to 'https://api.gdc.cancer.gov/legacy/projects/TCGA-COAD?expand=summary,summary.data_categories&pretty=true'
